In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv


In [2]:
data = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
data.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [4]:
data = data.fillna(0)

In [5]:
data.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [6]:
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,0.0,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [7]:
Gender = pd.get_dummies(data['gender'], drop_first=True)
Ever_Married = pd.get_dummies(data['ever_married'], drop_first=True)
Work_Type = pd.get_dummies(data['work_type'], drop_first=True)
residence_type = pd.get_dummies(data['Residence_type'], drop_first=True)
Smoking_Status = pd.get_dummies(data['smoking_status'], drop_first=True)

data = pd.concat([data,Gender,Ever_Married, Work_Type, residence_type, Smoking_Status],axis=1)
data = data.drop(['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis = 'columns')

In [8]:
data.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,Male,Other,Yes,Never_worked,Private,Self-employed,children,Urban,formerly smoked,never smoked,smokes
0,9046,67.0,0,1,228.69,36.6,1,1,0,1,0,1,0,0,1,1,0,0
1,51676,61.0,0,0,202.21,0.0,1,0,0,1,0,0,1,0,0,0,1,0
2,31112,80.0,0,1,105.92,32.5,1,1,0,1,0,1,0,0,0,0,1,0
3,60182,49.0,0,0,171.23,34.4,1,0,0,1,0,1,0,0,1,0,0,1
4,1665,79.0,1,0,174.12,24.0,1,0,0,1,0,0,1,0,0,0,1,0


In [9]:
X = data.drop('stroke', axis='columns')
y = data.stroke

In [10]:
X.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,Male,Other,Yes,Never_worked,Private,Self-employed,children,Urban,formerly smoked,never smoked,smokes
0,9046,67.0,0,1,228.69,36.6,1,0,1,0,1,0,0,1,1,0,0
1,51676,61.0,0,0,202.21,0.0,0,0,1,0,0,1,0,0,0,1,0
2,31112,80.0,0,1,105.92,32.5,1,0,1,0,1,0,0,0,0,1,0
3,60182,49.0,0,0,171.23,34.4,0,0,1,0,1,0,0,1,0,0,1
4,1665,79.0,1,0,174.12,24.0,0,0,1,0,0,1,0,0,0,1,0


In [11]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
X_train.shape

(3577, 17)

In [14]:
X_test.shape

(1533, 17)

## ANN Model with keras

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import matplotlib.pyplot as plt

In [16]:
model = Sequential()
# add first hidden layer with input diamension
model.add(Dense(units = 32, activation='relu', kernel_initializer = 'he_uniform', input_dim = 17))
# add second hidden layer
model.add(Dense(units = 16, activation='relu', kernel_initializer = 'he_uniform'))
# add output layer
model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'glorot_uniform'))

In [17]:
# now we compile the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# train the model
model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1)

Epoch 1/50
28/28 [==============================] - 1s 2ms/step - loss: 190.8086 - accuracy: 0.9059
Epoch 2/50
28/28 [==============================] - 0s 2ms/step - loss: 52.6838 - accuracy: 0.9362
Epoch 3/50
28/28 [==============================] - 0s 2ms/step - loss: 16.0776 - accuracy: 0.9106
Epoch 4/50
28/28 [==============================] - 0s 2ms/step - loss: 13.8401 - accuracy: 0.8401
Epoch 5/50
28/28 [==============================] - 0s 2ms/step - loss: 18.8502 - accuracy: 0.8968
Epoch 6/50
28/28 [==============================] - 0s 2ms/step - loss: 24.7428 - accuracy: 0.9294
Epoch 7/50
28/28 [==============================] - 0s 2ms/step - loss: 18.6291 - accuracy: 0.9062
Epoch 8/50
28/28 [==============================] - 0s 2ms/step - loss: 21.8305 - accuracy: 0.9396
Epoch 9/50
28/28 [==============================] - 0s 2ms/step - loss: 18.4539 - accuracy: 0.9106
Epoch 10/50
28/28 [==============================] - 0s 2ms/step - loss: 17.7905 - accuracy: 0.9291
Epoch 11

In [18]:
acc = model.evaluate(X_test, y_test)

48/48 [==============================] - 1s 1ms/step - loss: 15.3341 - accuracy: 0.9576


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                576       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,121
Trainable params: 1,121
Non-trainable params: 0
_________________________________________________________________


In [20]:
y_ann = model.predict(X_test)
y_ann = y_ann > 0.5

In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_ann, y_test)
cm

array([[1468,   63],
       [   2,    0]])

## Support Vector Machine (SVM)

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model_svm = SVC()
model_svm.fit(X_train, y_train)
y_svm = model_svm.predict(X_test)
acc_svm = accuracy_score(y_svm, y_test)
cm_svm = confusion_matrix(y_svm, y_test)
acc_svm

0.958904109589041

In [23]:
cm_svm

array([[1470,   63],
       [   0,    0]])

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_svm, y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1533
           1       0.00      0.00      0.00         0

    accuracy                           0.96      1533
   macro avg       0.50      0.48      0.49      1533
weighted avg       1.00      0.96      0.98      1533



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Random Forest Model

In [25]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
y_rf = model_rf.predict(X_test)
acc_rf = accuracy_score(y_rf, y_test)
cm_rf = confusion_matrix(y_rf, y_test)
acc_rf

0.95955642530985

In [26]:
cm_rf

array([[1470,   62],
       [   0,    1]])

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_rf, y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1532
           1       0.02      1.00      0.03         1

    accuracy                           0.96      1533
   macro avg       0.51      0.98      0.51      1533
weighted avg       1.00      0.96      0.98      1533



In [28]:
print("Artificial Neural Network Accuracy : ", acc)
print("Support Vector Machine Accuracy : ", acc_svm)
print("Random Forest Accuracy : ", acc_rf)

Artificial Neural Network Accuracy :  [15.334075927734375, 0.9575994610786438]
Support Vector Machine Accuracy :  0.958904109589041
Random Forest Accuracy :  0.95955642530985
